In [14]:
import pandas as pd
import numpy as np
import re as re
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
def extract_data_novo(url):
    
    
    """
    This function will collect the grant information provided from the Novo Nordisk foundation grant database with the url: https://novonordiskfonden.dk/en/grant-recipient/
    
    The paths are found using devtools, and found through the webpage.
    
    note: When the window opens, one must actively choose a web browser (eg. Ecosia) and close the
    the Cookie popup window - to start the extraction. 
    
    The information is saved in the lists of the following groups:
    receiver_l = []
    receiver_title_l = []
    description_l = []
    year_l = []
    amount_L = []
    location_l = []
    title_l = []
    
    and if data is not avaiable for a certain group, then "NA" is inserted. 
    
    The lists are finally zipped together in the biglist dataframe.
    
    input: target_url (https://novonordiskfonden.dk/en/grant-recipient/")
    return: Dataframe with text extracted
    """
    
    receiver_l = []
    receiver_title_l = []
    description_l = []
    year_l = []
    amount_L = []
    location_l = []
    title_l = []
    
    stored_description_lack_info = []
    
    
    driver.get(url)
    driver.maximize_window() #opening full web browser window
    time.sleep(5)
    
    #click on the #donations button
    action = ActionChains(driver)
    
    try:
        for page_num in range(1,360): #388 is full number (run again)
            """we click next page 360 times"""
            #time.sleep(5)
            
            #ActionChains(driver).move_to_element(driver.find_element(By.XPATH, "/html/body/div[2]/main/div/div/article/div[2]/div[1]/div/div/div[2]/table/tbody/tr[1]/td[1]")).perform()  #move to place
            for i in range(1,21):
                time.sleep(2)
                action.move_to_element(driver.find_element(By.CSS_SELECTOR, f"body > main > div.container > div > div.col-10.col-12-md > div.nnf-grant-list.nnf-grant-list--margin-top-24.text--blue > div.nnf-grant-list__items > div:nth-child({i}) > div.nnf-grant-list__owner.nnf-grant-list__info > div.nnf-grant-list__owner-name")).perform()  #move to the line of interest
                
                #recever name
                find_element_function(receiver_l, f"body > main > div.container > div > div.col-10.col-12-md > div.nnf-grant-list.nnf-grant-list--margin-top-24.text--blue > div.nnf-grant-list__items > div:nth-child({i}) > div.nnf-grant-list__owner.nnf-grant-list__info > div.nnf-grant-list__owner-name")
                
                #receiver proffesional title
                find_element_function(receiver_title_l, f"body > main > div.container > div > div.col-10.col-12-md > div.nnf-grant-list.nnf-grant-list--margin-top-24.text--blue > div.nnf-grant-list__items > div:nth-child({i}) > div.nnf-grant-list__owner.nnf-grant-list__info > div.nnf-grant-list__owner-title")
                
                #location
                find_element_function(location_l, f"body > main > div.container > div > div.col-10.col-12-md > div.nnf-grant-list.nnf-grant-list--margin-top-24.text--blue > div.nnf-grant-list__items > div:nth-child({i}) > div.nnf-grant-list__location.nnf-grant-list__info > div.nnf-grant-list__info-text")
                
                #year
                find_element_function(year_l, f"body > main > div.container > div > div.col-10.col-12-md > div.nnf-grant-list.nnf-grant-list--margin-top-24.text--blue > div.nnf-grant-list__items > div:nth-child({i}) > div.nnf-grant-list__year.nnf-grant-list__info > div.nnf-grant-list__info-text")
                
                #amount
                find_element_function(amount_L, f"body > main > div.container > div > div.col-10.col-12-md > div.nnf-grant-list.nnf-grant-list--margin-top-24.text--blue > div.nnf-grant-list__items > div:nth-child({i}) > div.nnf-grant-list__amount.nnf-grant-list__info > div.nnf-grant-list__info-text")
                
                #description
                find_element_function(description_l, f"body > main > div.container > div > div.col-10.col-12-md > div.nnf-grant-list.nnf-grant-list--margin-top-24.text--blue > div.nnf-grant-list__items > div:nth-child({i}) > div.nnf-grant-list__description.nnf-grant-list__info")


                if i == 20:
                    ActionChains(driver).move_to_element(driver.find_element(By.CSS_SELECTOR, "body > main > div.container > div > div.col-10.col-12-md > div.nnf-grant-list.nnf-grant-list--margin-top-24.text--blue > ul > li.nnf-pagination__nav.nnf-pagination__nav--next")).perform()  #move to click button
                    element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > main > div.container > div > div.col-10.col-12-md > div.nnf-grant-list.nnf-grant-list--margin-top-24.text--blue > ul > li.nnf-pagination__nav.nnf-pagination__nav--next")))
                    element.click()
                    print("page number number:", page_num)
                    time.sleep(5)
                
                    
            
    except NoSuchElementException: #avoid programme crashing
        print("Loop ended")

 

    df_final = pd.DataFrame(list(zip(year_l, receiver_l, amount_L, description_l, location_l, receiver_title_l)), columns=["Year",'Reciever','Grant size (DKK)', 'Description', "Location", "Receiver Proffession"])

    return df_final

def find_element_function(listname, css_selector_p):
    """
    helper function, that will first try to find the right element, 
    and if an error is raied, it will insert an "NA" instead
    """
    try:
        text_found = driver.find_element(By.CSS_SELECTOR, css_selector_p).text
        listname.append(text_found)
    except ElementClickInterceptedException:
        text_found = "NA"
        listname.append(location)
    except StaleElementReferenceException:
        text_found = "NA"
        listname.append(location)
    except NoSuchElementException:
        text_found = "NA"
        listname.append(text_found)
        
        return

biglist = extract_data_novo("https://novonordiskfonden.dk/en/grant-recipient/")
pd.set_option('display.max_columns', None)

biglist.to_csv('novo_data_24th_sep.csv', index=True)

page number number: 1
page number number: 2
page number number: 3
page number number: 4
page number number: 5
page number number: 6
page number number: 7
page number number: 8
page number number: 9
page number number: 10
page number number: 11
page number number: 12
page number number: 13
page number number: 14
page number number: 15
page number number: 16
page number number: 17
page number number: 18
page number number: 19
page number number: 20
page number number: 21
page number number: 22
page number number: 23
page number number: 24
page number number: 25
page number number: 26
page number number: 27
page number number: 28
page number number: 29
page number number: 30
page number number: 31
page number number: 32
page number number: 33
page number number: 34
page number number: 35
page number number: 36
page number number: 37
page number number: 38
page number number: 39
page number number: 40
page number number: 41
page number number: 42
page number number: 43
page number number: 

page number number: 347
page number number: 348
page number number: 349
page number number: 350
page number number: 351
page number number: 352
page number number: 353
page number number: 354
page number number: 355
page number number: 356
page number number: 357
page number number: 358
page number number: 359
